<a href="https://colab.research.google.com/github/abhinavpy/Tensorflow-Keras-Tutorials/blob/master/Early_Stopping_in_Keras_to_Prevent_Overfitting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Early stopping in Keras to Prevent Overfitting
Overfitting occurs when a neural network is trained to the point that it begins to memorize
rather than generalize

It is important to segment the original dataset into several datdasets:

1. Training Set
2. Validation Set
3. Holdout Set

There are several different ways that these sets can be constructed. The following programs demonstrate some of these.

The first method is a training and validation set. The training data are used to train the neural network until the validation set no longer improves. This attempt to stop at a near optimal training point. This method will only give accurate "out of sample" predictions for the validation set, this is usually 20% or so of the data. The predictions for the training data will be overly oprimistic, as these were the data that the neural network was trained on.

In [4]:
import pandas as pd
import io
import requests
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping

df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/iris.csv", na_values=['NA', '?'])

# Convert to numpy - Classification
x = df[['sepal_l', 'sepal_w', 'petal_l', 'petal_w']].values
dummies = pd.get_dummies(df['species']) # Classification
species = dummies.columns
y = dummies.values

# Split into validation and training sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

# Build neural network
model = Sequential()
model.add(Dense(50, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(25, activation='relu')) # Hidden 2
model.add(Dense(y.shape[1], activation='softmax')) # Output
model.compile(loss='categorical_crossentropy', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1,
                       mode='auto',restore_best_weights=True)
model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=[monitor],
         verbose=2, epochs=500)


Train on 112 samples, validate on 38 samples
Epoch 1/500
112/112 - 0s - loss: 1.2291 - val_loss: 1.0696
Epoch 2/500
112/112 - 0s - loss: 1.0807 - val_loss: 0.9657
Epoch 3/500
112/112 - 0s - loss: 0.9728 - val_loss: 0.9067
Epoch 4/500
112/112 - 0s - loss: 0.9178 - val_loss: 0.8739
Epoch 5/500
112/112 - 0s - loss: 0.8800 - val_loss: 0.8423
Epoch 6/500
112/112 - 0s - loss: 0.8475 - val_loss: 0.8087
Epoch 7/500
112/112 - 0s - loss: 0.8142 - val_loss: 0.7752
Epoch 8/500
112/112 - 0s - loss: 0.7818 - val_loss: 0.7396
Epoch 9/500
112/112 - 0s - loss: 0.7491 - val_loss: 0.7013
Epoch 10/500
112/112 - 0s - loss: 0.7182 - val_loss: 0.6648
Epoch 11/500
112/112 - 0s - loss: 0.6881 - val_loss: 0.6317
Epoch 12/500
112/112 - 0s - loss: 0.6597 - val_loss: 0.6010
Epoch 13/500
112/112 - 0s - loss: 0.6351 - val_loss: 0.5750
Epoch 14/500
112/112 - 0s - loss: 0.6107 - val_loss: 0.5520
Epoch 15/500
112/112 - 0s - loss: 0.5880 - val_loss: 0.5315
Epoch 16/500
112/112 - 0s - loss: 0.5652 - val_loss: 0.5113
Epoc

**_As you can see from above, the entire number of requested epochs were not used. The neural network training stopped once the validation set no lonnger improved._**

In [5]:
from sklearn.metrics import accuracy_score
pred = model.predict(x_test)
predict_classes = np.argmax(pred, axis = 1)
expected_classes = np.argmax(y_test, axis = 1)
correct = accuracy_score(expected_classes, predict_classes)
print(f"Accuracy: {correct}")

Accuracy: 1.0


## Early stopping with Regression

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

df = pd.read_csv("https://data.heatonresearch.com/data/t81-558/auto-mpg.csv",
                na_values=['NA', '?'])
cars = df['name']

# Handle missing value
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# pandas to numpy
x = df[['cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin']].values
y = df['mpg'].values # regression

# Split into validation and training sets
x_train, y_test, y_train, y_test = train_test_split(x,y,test_size=0.25, random_state=42)

#Build the neural network 
model = Sequential()
model.add(Dense(25, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(1)) # Output
model.compile(loss = 'mean_squared_error', optimizer='adam')

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose = 1,
                       mode = 'auto', restore_best_weights = True)
model.fit(x_train, y_train, validation_data=(x_test, y_test), callbacks=[monitor], verbose=2, epochs=1000)

ValueError: ignored

In [12]:
# Measure RMSE Error. RMSE is common for regression
pred = model.predict(x_test)
score = np.sqrt(metrics.mean_squared_error(pred, y_test))
print(f"Final score: (RMSE): {score}")

ValueError: ignored